In [ ]:
from typing import List, Optional, Literal,Annotated, TypedDict
from pydantic import BaseModel
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, AnyMessage
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from langgraph.types import interrupt, Command

llm = init_chat_model(model="openai:gpt-4o")

QUESTIONS = [
    "What is the name of the competitor?",
    "Whats your favorite color?",
    "Pick a number between 1 and 100",
    "What is your name?"
]

class CompetitiveAnalysisState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    user_answered_all_questions: bool


In [ ]:
system_prompt = """
You need to verify the user answered all the questions and if not, ask the next question.
The list of questions is:
{QUESTIONS}

User the history of messages to verify if the user answered all the questions.
"""

class next_question(BaseModel):
    question: str
    answered_all_questions: bool

In [ ]:
def CompetitiveAnalysisNode(state: CompetitiveAnalysisState):
    system_message = SystemMessage(content=system_prompt)
    history = state['messages']
    messages = [system_message] + history
    llm_with_structured_output = llm.with_structured_output(next_question)
    ai_message = llm_with_structured_output.invoke(messages)

    user_response = interrupt({
        "question": ai_message,
    })
    messages.append(AIMessage(content=ai_message))
    messages.append(HumanMessage(content=user_response))

    return {"messages": messages}





builder = StateGraph(CompetitiveAnalysisState)

builder.add_node("CompetitiveAnalysisNode", CompetitiveAnalysisNode)
builder.add_edge(START, "CompetitiveAnalysisNode")
builder.add_edge("CompetitiveAnalysisNode", END)

graph = builder.compile()
